In [3]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "2 + 2 ", "output": "4"},
    {"input": "3 * 3 ", "output": "9"},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

few_short_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

print(few_short_prompt.format())

Human: 2 + 2 
AI: 4
Human: 3 * 3 
AI: 9


In [5]:
# 组装成最终的模板，并与大模型结合使用
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个数学天才，请根据以下问题和答案，回答问题："),
    few_short_prompt,
    ("human", "{input}"),
])

print(final_prompt.format(input="13 + 13 = "))

System: 你是一个数学天才，请根据以下问题和答案，回答问题：
Human: 2 + 2 
AI: 4
Human: 3 * 3 
AI: 9
Human: 13 + 13 = 


In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

api_key = "xxx"
base_url = "http://localhost:1234/v1"

chat = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.7)

output_parser = StrOutputParser()

chain = final_prompt | chat | output_parser

print(chain.invoke({"input": "12的平方是多少"}))


144


In [9]:
chat.invoke(input = "12的平方是多少")

AIMessage(content='12的平方是144。计算方法为 \\(12 \\times 12 = 144\\)。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 34, 'total_tokens': 61, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-32b-instruct', 'system_fingerprint': 'qwen2.5-32b-instruct', 'finish_reason': 'stop', 'logprobs': None}, id='run-bc52baa6-6b27-4c2b-9cdf-46ee60e2dedf-0', usage_metadata={'input_tokens': 34, 'output_tokens': 27, 'total_tokens': 61, 'input_token_details': {}, 'output_token_details': {}})

## 动态几次提示

有时您可能希望根据输入来限制显示哪些示例。为此，您可以将 examples 替换为 example_selector 。其他组件与上面相同！回顾一下，动态几次提示模板将如下所示：

- example_selector ：负责为给定输入选择少数样本（以及它们返回的顺序）。它们实现了 BaseExampleSelector 接口。一个常见的例子是向量存储支持的 SemanticSimilarityExampleSelector

- example_prompt ：通过其 format_messages 方法将每个示例转换为 1 条或多条消息。一个常见的示例是将每个示例转换为一条人工消息和一条人工智能消息响应，或者一条人工消息后跟一条函数调用消息。

这些可以再次与其他消息和聊天模板组合以组合您的最终提示。

In [11]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="/Users/libing/kk_LLMs/bge-large-zh-v1.5")

In [12]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "牛对月亮说了什么？", "output": "什么都没有"},
    {
        "input": "给我写一首关于月亮的五言诗",
        "output": "月儿挂枝头，清辉洒人间。 银盘如明镜，照亮夜归人。 思绪随风舞，共赏中秋圆。",
    },
]

to_verctorize = [" ".join(example.values()) for example in examples]
vectorstore = Chroma.from_texts(to_verctorize, embeddings, metadatas=examples)

In [14]:
example_selector = SemanticSimilarityExampleSelector(vectorstore=vectorstore, k=1)

example_selector.select_examples({"input": "对牛弹琴"})

[{'input': '牛对月亮说了什么？', 'output': '什么都没有'}]

In [15]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

few_short_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_prompt = example_prompt,
    example_selector = example_selector,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个数学天才，请根据以下问题和答案，回答问题："),
    few_short_prompt,
    ("human", "{input}"),
])

print(final_prompt.format(input="9 * 2等于多少？"))

System: 你是一个数学天才，请根据以下问题和答案，回答问题：
Human: 2+3
AI: 5
Human: 9 * 2等于多少？


In [16]:
chain = final_prompt | chat | output_parser

chain.invoke({"input": "9 * 2等于多少？"})

'9 * 2 = 18'

In [18]:
chat.invoke(input = "9 * 2")

AIMessage(content='9 * 2 = 18', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 33, 'total_tokens': 41, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-32b-instruct', 'system_fingerprint': 'qwen2.5-32b-instruct', 'finish_reason': 'stop', 'logprobs': None}, id='run-a905aa9d-2b19-4375-b52d-148e84efa3f4-0', usage_metadata={'input_tokens': 33, 'output_tokens': 8, 'total_tokens': 41, 'input_token_details': {}, 'output_token_details': {}})